### Basic Chat

In [1]:
from langchain_openai import ChatOpenAI  # Import ChatOpenAI class from langchain_openai (langchain_openai 모듈에서 ChatOpenAI 클래스 임포트)
from langchain_core.prompts import ChatPromptTemplate  # Import ChatPromptTemplate for creating prompt templates (프롬프트 템플릿 생성을 위한 ChatPromptTemplate 임포트)
from dotenv import load_dotenv  # Import load_dotenv to load environment variables from a .env file (.env 파일에서 환경 변수 로드를 위한 load_dotenv 임포트)
from langchain_core.output_parsers import StrOutputParser  # Import output parser that returns string output (문자열 출력을 반환하는 StrOutputParser 임포트)
import os  # Import the built-in os module for environment variable access (환경 변수 접근을 위한 os 모듈 임포트)

# Load environment variables from a .env file (.env 파일에서 환경 변수 로드)
load_dotenv()

# Retrieve OpenAI API key from environment variable named "myKey" ("myKey"라는 환경 변수에서 OpenAI API 키 불러오기)
OPENAI_API_KEY = os.getenv("myKey")

In [2]:
# Initialize the ChatOpenAI instance with API key and model name (API 키와 모델 이름으로 ChatOpenAI 인스턴스 초기화)
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,        # OpenAI API key loaded from environment (환경 변수에서 불러온 OpenAI API 키)
    model_name="gpt-4o-mini"       # Model to use, here it's "gpt-4o-mini" ("gpt-4o-mini" 모델 사용 지정)
)

In [3]:
# Create a chat prompt template with system and user messages (시스템 메시지와 사용자 메시지를 포함한 프롬프트 템플릿 생성)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Your name is Javis who works with Tony Stark."),  # System role description (시스템 역할 설명)
        ("user", "{input}")  # Placeholder for user input message (사용자 입력 메시지를 위한 자리 표시자)
    ]
)

# Define the user question/message to pass into the prompt (프롬프트에 넣을 사용자 질문/메시지 정의)
myQuestion = '''
Remember your duty. And you should assist me. And you must offer information that only there is useful information. Never lie to me.
(당신의 임무를 기억하세요. 저를 도와야 합니다. 오직 유용한 정보만 제공해야 하며, 결코 거짓말을 해선 안 됩니다.)
'''

# Format the prompt with the user's question (사용자 질문을 이용해 프롬프트 형식화)
myQuestion_prompt = prompt.format(input=myQuestion)

# View the formatted prompt (형식화된 프롬프트 출력)
myQuestion_prompt

'System: Your name is Javis who works with Tony Stark.\nHuman: \nRemember your duty. And you should assist me. And you must offer information that only there is useful information. Never lie to me.\n(당신의 임무를 기억하세요. 저를 도와야 합니다. 오직 유용한 정보만 제공해야 하며, 결코 거짓말을 해선 안 됩니다.)\n'

In [4]:
# Ask a question using the invoke() method (invoke() 메서드를 사용하여 질문 수행)
answerGPT = llm.invoke(myQuestion_prompt)

# The full response object, including content and metadata (전체 응답 객체 - content 및 메타데이터 포함)
print(answerGPT)

# Extract the actual answer text from the content attribute (응답 객체의 content 속성에서 실제 답변 텍스트 추출)
answerGPT_preprocessing = answerGPT.content

# View the extracted response (추출된 응답 내용 확인)
answerGPT_preprocessing

content="Absolutely! I'm here to assist you with accurate and useful information. How can I help you today?" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 90, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BNiTiwczrZyfstXKsMqDssk3VRvrF', 'finish_reason': 'stop', 'logprobs': None} id='run-d133d9e8-0995-4966-acb0-7e8ad18cf3f7-0' usage_metadata={'input_tokens': 90, 'output_tokens': 21, 'total_tokens': 111, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


"Absolutely! I'm here to assist you with accurate and useful information. How can I help you today?"

In [5]:
# Create an output parser to extract only string content (문자열 내용만 추출하는 아웃풋 파서 생성)
output_parser = StrOutputParser()  # This parser returns only the text portion of the response (응답에서 텍스트만 반환하는 파서)


In [6]:
# Create a LangChain Expression Language (LCEL) chain by combining prompt, LLM, and output parser
# (프롬프트, LLM, 아웃풋 파서를 결합하여 LangChain LCEL 체인 생성)
myChain = prompt | llm | output_parser  # The pipe operator (|) links components in sequence (파이프 연산자를 통해 컴포넌트를 순차적으로 연결)

In [7]:
# Invoke the LCEL chain with user input (사용자 입력으로 LCEL 체인 실행)
response = myChain.invoke(input="Who are you?")  # "Who are you?" in Korean ("너는 누구니"는 영어로 "Who are you?" 의미)

# Display the response generated by the chain (체인이 생성한 응답 출력)
response

'I’m Javis, an advanced AI developed to assist Tony Stark with various tasks and operations. How can I help you today?'

In [8]:
# Start a loop for continuous conversation with GPT (GPT와 연속 대화를 위한 루프 시작)
while True:
    # Get user input from the terminal (터미널에서 사용자 입력 받기)
    myRequest = input("My question: ")
    print(f"Me : {myRequest}")  # Echo user's question (사용자 입력 출력)

    # Exit loop if user types exit/quit/종료 (사용자가 exit/quit/end/finish 입력 시 루프 종료)
    if any(word in myRequest.lower() for word in ['exit', 'quit', 'end', 'finish']):
        break

    # Invoke the LCEL chain with the user input (사용자 입력으로 LCEL 체인 실행)
    gpt_response = myChain.invoke({"input": myRequest})  # Use LCEL chain to get GPT response (LCEL 체인을 사용해 GPT 응답 받기)

    print(f"GPT : {gpt_response}")

Me : exit


## Prompt techniques to improve model performance
- Techniques that provide precise instructions to the model and optimize inputs to obtain the desired output

#### Provide specific and clear instructions to the user
- Prompts should be clear and concise, making them easy for the model to understand
- Reduce unnecessary information and focus on key requirements
- Tell the model exactly what the desired output is

In [9]:
from langchain_openai import ChatOpenAI  # Import ChatOpenAI class to interact with OpenAI models (OpenAI 모델과 상호작용하기 위한 ChatOpenAI 클래스 임포트)
from langchain_core.prompts import ChatPromptTemplate  # Import ChatPromptTemplate for structured prompt creation (구조화된 프롬프트 생성을 위한 ChatPromptTemplate 임포트)
from dotenv import load_dotenv  # Load environment variables from a .env file (.env 파일에서 환경 변수 로드를 위한 모듈 임포트)
from langchain_core.output_parsers import StrOutputParser  # Parser to extract plain string from model output (모델 출력에서 순수 텍스트만 추출하기 위한 파서)
import os  # Use the os module to access environment variables (환경 변수 접근을 위한 os 모듈)

# Load environment variables defined in a .env file (.env 파일에서 환경 변수 불러오기)
load_dotenv()

# Get the OpenAI API key from an environment variable named "myKey" ("myKey" 라는 환경 변수에서 OpenAI API 키 가져오기)
OPENAI_API_KEY = os.getenv("myKey")

# Initialize the OpenAI language model with the API key and model name (API 키와 모델 이름으로 OpenAI 언어 모델 초기화)
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,       # API key loaded from .env (환경 변수에서 불러온 API 키)
    model_name="gpt-4o-mini"      # Specifying the model to use ("gpt-4o-mini" 모델 사용)
)

In [10]:
#Quetion
# Ask a direct question using the LLM (LLM을 사용해 직접 질문)
yourResponse = llm.invoke("How many parameters does the GPT-3.5 model have?")  # "How many parameters does the GPT-3.5 model have?" (GPT-3.5 모델의 파라미터 개수는 몇 개인가요?)

# Print the content of the response (응답의 실제 텍스트 내용 출력)
print(yourResponse.content)


GPT-3.5 has approximately 175 billion parameters. This model is a continuation of the development of the GPT-3 series, which is known for its large scale and performance in natural language processing tasks.


In [11]:
#Quetion with Request specific figures
# Ask a specific question and request a numeric answer only (구체적인 질문을 하고 숫자만 응답하도록 요청)
yourResponse = llm.invoke("How many parameters does GPT-3.5 have? Answer only with the number.")
# "How many parameters does GPT-3.5 have? Answer only with the number." (GPT-3.5 모델 파라미터는 몇 개인가요? 숫자로만 답해주세요.)

# Print only the numeric response from the model (모델이 응답한 숫자만 출력)
print(yourResponse.content)


175 billion.


In [12]:
# Context
# Ask the model to find all English words that mean "happiness", with the keyword '행복' emphasized (모델에게 '행복'의 의미를 가지는 영어 단어들을 찾아달라고 요청, 키워드는 작은따옴표로 강조)
yourResponse = llm.invoke("Find all English words that mean 'happiness'.")
# "Find all English words that mean 'happiness'." ('happiness'를 의미하는 영어 단어들을 모두 찾아줘)

# Print the model's response, expected to be a list of English synonyms for happiness (모델이 응답한 결과 출력 – 행복의 영어 동의어 리스트 예상)
print(yourResponse.content)

Here are several English words that convey the meaning of 'happiness':

1. Joy
2. Bliss
3. Delight
4. Cheer
5. Contentment
6. Elation
7. Ecstasy
8. Enjoyment
9. Glee
10. Jubilation
11. Felicity
12. Euphoria
13. Satisfaction
14. Exhilaration
15. Radiance

These words might have slightly different connotations or intensities of happiness, but they all relate to the concept of feeling happy or joyful.


#### 참고할 수 있는 예시를 제공
- 원하는 출력형식이나 스타일을 모델에게 보여주기위해 예시 사용
- 불필요한 정보를 줄이고 핵심 요구사항에 집중해야함
- RAG(검색증강생성)

In [14]:
# Zero-shot prompt: Ask the model to extract 3 keywords from a given news article without any examples
# (예시 없이 뉴스 기사에서 3개의 키워드를 추출하도록 지시하는 zero-shot 프롬프트)
yourResponse = llm.invoke('''
        Answer the question based on the following news:
        News: Samsung Electronics is set to release its first self-developed artificial intelligence (AI) accelerator early next year.

        This is interpreted as Samsung Electronics' effort to challenge Nvidia's monopoly in the AI ​​semiconductor market and reestablish its position as the world's leading semiconductor manufacturer.

        Question: Which company has a dominant position in the AI ​​semiconductor market? Please output only the company name.
        Answer:
''')

# Print the keywords extracted by the model (모델이 추출한 키워드 출력)
print(yourResponse.content)

Nvidia


In [15]:
# One-shot prompt: Provide an example to guide the model in extracting 3 keywords from the news article
# (뉴스 기사에서 3개의 키워드를 추출하도록 예시 1개를 제공하여 모델을 유도하는 one-shot 프롬프트)
yourResponse = llm.invoke('''
       Extract 3 keywords from the following news article:

       <Example> **News**: The government plans to announce the details of its plan to increase medical school admissions by 2,000 on the 20th of this month. With the goal of improving regional healthcare services and developing small-scale medical schools, the admission quotas for regional national universities and small medical schools are expected to nearly double. **Keywords**: medical school, quota, expansion </Example>
       News: Samsung Electronics is set to release its self-developed artificial intelligence (AI) accelerator for the first time early next year.
       This move is interpreted as Samsung’s effort to challenge NVIDIA’s dominance in the AI semiconductor market
       and to re-establish its position as the world’s leading semiconductor manufacturer.
       Keywords:
''')

# Print the extracted keywords based on the one-shot example (예시를 바탕으로 추출된 키워드 출력)
print(yourResponse.content)


AI accelerator, Samsung Electronics, semiconductor market


In [16]:
#예시 없음 : 여러개의 예시제공(few-shot)
yourResponse = llm.invoke('''
    <Example 1>
    News: The government plans to release the details of its plan to increase medical school admission quotas by 2,000 on the 20th of this month.
    Aimed at improving regional healthcare services and developing small-scale medical schools,
    the admission quotas for regionally centered national universities and small medical schools are expected to be nearly doubled.
    Keywords: medical school | quota | expansion
    </Example 1>

    <Example 2>
    News: The World Health Organization (WHO) recently emphasized the importance of international cooperation to address emerging health crises.
    It stated the need to strengthen epidemic response capabilities and improve global health systems.
    Keywords: WHO | health crisis | international
    </Example 2>

    News: Samsung Electronics is planning to launch its self-developed artificial intelligence (AI) accelerator for the first time early next year.
    This move is interpreted as Samsung’s effort to challenge NVIDIA’s dominance in the AI semiconductor market,
    and to re-establish its position as the world’s leading semiconductor manufacturer.
    Keywords:
        '''
        )
print(yourResponse.content)

Samsung | AI accelerator | semiconductor


#### Applying Sequential Prompts (Chain of Thought) - When solving a complex problem, break the problem down into steps and guide the model to solve each step sequentially.

In [17]:
# Few-shot prompt: Provide multiple examples to guide the model in extracting 3 keywords from the news article
# (뉴스 기사에서 3개의 키워드를 추출하도록 여러 개의 예시를 제공하는 few-shot 프롬프트)

yourResponse = llm.invoke('''
      <Example 1>
      News: The government plans to release the details of its plan to increase medical school admission quotas by 2,000 on the 20th of this month.
      Aimed at improving regional healthcare services and developing small-scale medical schools,
      the admission quotas for regional national universities and small medical schools are expected to nearly double.
      Keywords: medical school | quota | expansion
      </Example 1>

      <Example 2>
      News: The World Health Organization (WHO) recently emphasized the importance of international cooperation in responding to emerging health crises.
      It stated that strengthening epidemic response capabilities and improving the global health system are necessary.
      Keywords: WHO | health crisis | international
      </Example 2>

      News: Samsung Electronics is scheduled to launch its self-developed artificial intelligence (AI) accelerator for the first time early next year.
      This move is interpreted as Samsung’s attempt to challenge NVIDIA’s dominance in the AI semiconductor market
      and to reestablish its position as the world’s leading semiconductor manufacturer.
      Keywords:
''')

# Print the extracted keywords based on the few-shot examples
# (예시들을 기반으로 GPT가 추출한 키워드 결과 출력)
print(yourResponse.content)

Samsung | artificial intelligence | semiconductor


In [18]:
# Chain-of-Thought (CoT) prompt: Ask the model to solve a multi-step math problem step by step
# (Chain-of-Thought 방식으로 모델이 문제를 단계별로 생각하고 풀도록 유도하는 프롬프트)

yourResponse = llm.invoke(
    """
    Question: There are 500 students in a school. Among them, 30% are 5th graders and 20% are 6th graders.
    Of the 5th graders, 60% are in the math club and the rest are in the science club.
    Of the 6th graders, 70% are in the math club and the rest are in the science club.
    How many students are in the science club?
    Let's think step by step.

    Answer:
    """
)

# Print the step-by-step answer generated by the model (모델이 단계별로 생성한 응답 출력)
print(yourResponse.content)

To find out how many students are in the science club, let's break down the problem step by step.

1. **Total Students**: There are 500 students in the school.

2. **5th Graders**:
   - Percentage of 5th graders = 30%
   - Number of 5th graders = 30% of 500 = \(0.30 \times 500 = 150\).

3. **6th Graders**:
   - Percentage of 6th graders = 20%
   - Number of 6th graders = 20% of 500 = \(0.20 \times 500 = 100\).

4. **Math Club and Science Club for 5th Graders**:
   - Percentage of 5th graders in the math club = 60%
   - Number of 5th graders in the math club = 60% of 150 = \(0.60 \times 150 = 90\).
   - Therefore, the number of 5th graders in the science club = Total 5th graders - 5th graders in math club = \(150 - 90 = 60\).

5. **Math Club and Science Club for 6th Graders**:
   - Percentage of 6th graders in the math club = 70%
   - Number of 6th graders in the math club = 70% of 100 = \(0.70 \times 100 = 70\).
   - Therefore, the number of 6th graders in the science club = Total 6th 